#### From the IBL database, get data on weight and trial counts under different water regimes at CSHL
Anne Urai, CSHL, 2019

In [1]:
# PYTHON STARTS BY IMPORTING 'MODULES' AND 'PACKAGES' THAT WE'LL USE LATER

# GENERAL THINGS FOR COMPUTING AND PLOTTING
import pandas as pd
import numpy as np
import os, sys, time

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid", context="paper")

# set a path where you want figures to be saved
figpath  = os.path.join(os.path.expanduser('~'), 'Data/Figures_IBL')

# ibl specific things
import datajoint as dj
from ibl_pipeline import reference, subject, action, acquisition, data, behavior

Connecting anneurai@datajoint.internationalbrainlab.org:3306


If your login to DataJoint was successful, you should see something like 

'Connecting youruser@datajoint.internationalbrainlab.org:3306'

In [33]:
# FIND THE DATA WE NEED

# FIRST, WHICH SUBJECTS ARE DOING THIS CA EXPERIMENT?
subj = (subject.Subject() - subject.Death & 'subject_birth_date > "2018-09-01"').proj('subject_nickname', 'sex') * \
    (subject.SubjectLab() & 'lab_name="churchlandlab"').proj()
print(subj)

*subject_uuid  subject_nickna sex    
+------------+ +------------+ +-----+
0936a724-2151- CSHL_007       F      
19cf7867-e2d2- CSHL_010       F      
4c9577b3-bdd5- CSHL_002       M      
4f091667-90b9- CSHL_008       F      
535ae292-7006- CSHL_009       F      
5848e7f4-447d- CSHL_006       F      
6e1e46eb-a742- CSHL_003       M      
8f6fc7ce-4de0- CSHL_004       M      
ac7dfc1e-0c3c- CSHL_005       M      
ecba4914-30c3- CSHL_001       M      
 (Total: 10)



In [76]:
# SECOND, WHAT KIND OF DATA DO WE WANT FROM THEM?
# weight = pd.DataFrame((action.Weighing * subj).fetch(as_dict=True))
# water  = pd.DataFrame((action.WaterAdministration * subj).fetch(as_dict=True))
    
# GET FOR EACH DAY THE ANIMAL'S WEIGHT AND TRIAL COUNT
weight_info = action.Weighing.proj('weight', session_date='DATE(weighing_time)') 
weight_matched = (dj.U('subject_uuid', 'session_date') & weight_info) * weight_info

# TODO: average the weights per day!
# weights_average = weight_matched.aggr(avg_weight='MEAN(session_date)')

# do the same for water
# also return the day of the week, where 1 = Sunday
water_info = action.WaterAdministration.proj('watertype_name', 'adlib', 'administration_time', 'water_administered',
                                             session_date='DATE(administration_time)', water_admin_dow='dayofweek(administration_time)')
water_matched = (dj.U('subject_uuid', 'session_date') & water_info) * water_info
# TODO: sum the water counts per day, and give a list of all water types (unique)

# now get those two together with the trial counts
b = (behavior.TrialSet.proj('n_trials')) \
    * (acquisition.Session.proj(session_date='DATE(session_start_time)') 
       & 'session_date > "2019-03-01"') \
    * subj * weight_matched * water_matched
b


subject_uuid,session_start_time start time,session_date calculated attribute,weighing_time date time,administration_time date time,n_trials total trial numbers in this set,subject_nickname nickname,sex sex,weight weight,water_administered water administered,adlib,watertype_name,water_admin_dow calculated attribute
0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-04 15:27:54,2019-03-04,2019-03-04 14:56:00,2019-03-04 16:14:21,245,CSHL_007,F,18.94,0.252,0,Water 10% Sucrose,2
0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-04 15:27:54,2019-03-04,2019-03-04 15:27:54,2019-03-04 16:14:21,245,CSHL_007,F,18.93,0.252,0,Water 10% Sucrose,2
0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-05 15:48:54,2019-03-05,2019-03-05 14:05:00,2019-03-05 16:37:07,314,CSHL_007,F,18.71,0.7037,0,Water 10% Sucrose,3
0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-05 15:48:54,2019-03-05,2019-03-05 15:48:54,2019-03-05 16:37:07,314,CSHL_007,F,18.64,0.7037,0,Water 10% Sucrose,3
0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-05 15:48:54,2019-03-05,2019-03-05 17:36:00,2019-03-05 16:37:07,314,CSHL_007,F,20.55,0.7037,0,Water 10% Sucrose,3
0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-06 14:16:57,2019-03-06,2019-03-06 14:07:00,2019-03-06 15:08:13,322,CSHL_007,F,19.22,0.6784,0,Water 10% Sucrose,4
0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-06 14:16:57,2019-03-06,2019-03-06 14:16:57,2019-03-06 15:08:13,322,CSHL_007,F,19.22,0.6784,0,Water 10% Sucrose,4
0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-11 11:59:37,2019-03-11,2019-03-11 09:14:00,2019-03-11 12:45:06,382,CSHL_007,F,18.38,0.7953,0,Water 10% Sucrose,2
0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-11 11:59:37,2019-03-11,2019-03-11 11:59:37,2019-03-11 12:45:06,382,CSHL_007,F,18.83,0.7953,0,Water 10% Sucrose,2
0936a724-2151-446f-98b2-92c3e9dd36dc,2019-03-12 14:56:49,2019-03-12,2019-03-12 14:56:49,2019-03-12 16:15:31,726,CSHL_007,F,18.39,1.5872,0,Water 10% Sucrose,3
